<a href="https://colab.research.google.com/github/leenago/Asiae_edu/blob/master/Industrial_Bigdata/200923_NLPtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP의 TF-IDF 원리 알아보기

##1.준비하기

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import konlpy
from konlpy.tag import Kkma
kkma = Kkma()

In [ ]:
kkma.pos('안녕하세요,여러분')

[('안녕', 'NNG'), ('하세', 'NNG'), ('요', 'JX'), (',', 'SP'), ('여러분', 'NP')]

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

##2.웹크롤링 - 네이버 뉴스 데이터 수집

In [ ]:
url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={0}'
#query 부분 제거

src = requests.get(url.format('미국+기술주'))
html = src.content
len(html)

627183

In [ ]:
soup = bs(html, 'html.parser')
tgts = soup.select("ul.type01 li") #여기서 에러가 났음
len(tgts)

29

In [ ]:
for tgt in tgts:
    if (tgt.select('dl > dt')):
        headtxt = tgt.select('dl > dt')[0].text
        print(headtxt)

미국 증시, 3대 지수 상승…테슬라, 배터리데이 앞두고 급락
뉴욕증시, 코로나19 우려 속에서도 기술주 강세로 상승 출발
니콜라가 몰고온 기술주 먹구름···테슬라가 걷어낼까
대신증권, '미국 기술주 진단' 유튜브 생방송
MAGA 일제히 반등...뉴욕 증시, 기술주 주도로 상승
[뉴욕증시 마감] 코로나19 재확산에도 아마존·애플 기술주 활약에 반등
뉴욕증시, 기술주 활약에 동반 상승…나스닥 1.7%↑
뉴욕증시, 코로나19 우려에도 기술주 강세…다우 0.52% 상승 마감
[뉴욕마감]나스닥 1.7% 반등…아마존 주도 기술주 랠리
뉴욕증시, 코로나19 우려에도 기술주 강세...다우, 0.52% 상승 마감


In [ ]:
titles =[]

for tgt in tgts:
    if (tgt.select('dl > dt')):
        origin_addr = tgt.select('dl > dt a')[0].attrs['href'] #dl 안에 dt 안에 a
        main_title = tgt.select('dl > dt a')[0].attrs['title']
        titles.append(main_title)
        print(origin_addr,":",main_title)


#관련 기사들은 링크를 타고 들어가서 봐야한다.
#title

http://www.wowtv.co.kr/NewsCenter/News/Read?articleId=A202009230010&t=NN : 미국 증시, 3대 지수 상승…테슬라, 배터리데이 앞두고 급락
https://news.joins.com/article/olink/23473095 : 뉴욕증시, 코로나19 우려 속에서도 기술주 강세로 상승 출발
https://www.sedaily.com/NewsView/1Z7XXYWTAN : 니콜라가 몰고온 기술주 먹구름···테슬라가 걷어낼까
http://www.shinailbo.co.kr/news/articleView.html?idxno=1323707 : 대신증권, '미국 기술주 진단' 유튜브 생방송
http://www.bloter.net/archives/417466 : MAGA 일제히 반등...뉴욕 증시, 기술주 주도로 상승
http://www.ajunews.com/view/20200923060155848 : [뉴욕증시 마감] 코로나19 재확산에도 아마존·애플 기술주 활약에 반등
https://www.hankyung.com/finance/article/2020092395727 : 뉴욕증시, 기술주 활약에 동반 상승…나스닥 1.7%↑
http://mbn.mk.co.kr/pages/news/newsView.php?category=mbn00003&news_seq_no=4289265 : 뉴욕증시, 코로나19 우려에도 기술주 강세…다우 0.52% 상승 마감
https://www.news1.kr/articles/?4067167 : [뉴욕마감]나스닥 1.7% 반등…아마존 주도 기술주 랠리
https://www.ytn.co.kr/_ln/0104_202009230601344159 : 뉴욕증시, 코로나19 우려에도 기술주 강세...다우, 0.52% 상승 마감


##3.수집데이터에 pos tagging 하기

In [ ]:
worddict = {}

for t in titles:
    kpos = kkma.pos(t)
    
    jlist = []
    for pos in kpos:
        j='|'.join(pos)
        if(j in worddict.keys()):
            worddict[j] += 1
        else:
            worddict[j] = 1
print(worddict)
#'미국'이라는 단어가 2개, '증시'라는 단어가 7개 나왔다는 뜻. 

{'미국|NNP': 2, '증시|NNG': 7, ',|SP': 9, '3|NR': 1, '대|NNM': 1, '지수|NNG': 1, '상승|NNG': 6, '…|SE': 4, '테슬라|NNG': 2, '배터리|NNG': 1, '데이|NNG': 1, '앞|NNG': 1, '두고|NNP': 1, '급락|NNG': 1, '뉴욕|NNP': 7, '코로나|NNG': 4, '19|NR': 4, '우려|NNG': 3, '속|NNG': 1, '에서|JKM': 1, '도|JX': 4, '기술|NNG': 9, '주|NNG': 8, '강세|NNG': 3, '로|JKM': 1, '출발|NNG': 1, '니콜라|NNG': 1, '가|JKS': 2, '몰|VV': 1, '고|ECE': 1, '오|VV': 1, 'ㄴ|ETD': 1, '먹구름|NNG': 1, '···|SW': 1, '걷어내|VV': 1, 'ㄹ까|EFQ': 1, '대신|NNG': 1, '증권|NNG': 1, "'|SS": 2, '주|XSN': 1, '진단|NNG': 1, '유|NNG': 1, '튜브|NNG': 1, '생방송|NNG': 1, 'MAGA|OL': 1, '일제히|MAG': 1, '반등|NNG': 2, '...|SE': 2, '주도로|NNG': 1, '[|SS': 2, '마감|NNG': 4, ']|SS': 2, '재|XPN': 1, '확산|NNG': 1, '에|JKM': 5, '아마존|NNG': 1, '·|SP': 1, '애플|NNP': 1, '활약|NNG': 2, '동반|NNG': 1, '나스닥|NNG': 2, '1.7|NR': 2, '%|SW': 4, '↑|SW': 1, '다|MAG': 2, '우|NNG': 2, '0.52|NR': 2, '반|NNG': 1, '등|NNB': 1, '아마|MAG': 1, '존|NNP': 1, '주도|NNG': 1, '랠리|NNG': 1}


- NLP는 정규식을 잘 해야한다!
  - [정규식페이지](https://regexr.com/)

In [ ]:
#정규식으로 한글만 뽑아내기
import re
pattern =r'[^가-힣]'
titles2 =[]

for tgt in tgts:
    if (tgt.select('dl > dt')):
        origin_addr = tgt.select('dl > dt a')[0].attrs['href'] #dl 안에 dt 안에 a
        main_title = tgt.select('dl > dt a')[0].attrs['title']
        rtitle = re.sub(pattern, ' ', main_title) #정규식에서 빼고 싶은 단어들의 자리를 공백처리
        titles2.append(rtitle)
print(titles2)

['미국 증시   대 지수 상승 테슬라  배터리데이 앞두고 급락', '뉴욕증시  코로나   우려 속에서도 기술주 강세로 상승 출발', '니콜라가 몰고온 기술주 먹구름   테슬라가 걷어낼까', '대신증권   미국 기술주 진단  유튜브 생방송', '     일제히 반등   뉴욕 증시  기술주 주도로 상승', ' 뉴욕증시 마감  코로나   재확산에도 아마존 애플 기술주 활약에 반등', '뉴욕증시  기술주 활약에 동반 상승 나스닥      ', '뉴욕증시  코로나   우려에도 기술주 강세 다우       상승 마감', ' 뉴욕마감 나스닥      반등 아마존 주도 기술주 랠리', '뉴욕증시  코로나   우려에도 기술주 강세   다우        상승 마감']


In [ ]:
#제일 많이 나오는 단어 위주로 뽑기
#해당하는 word map을 만들고, 의미있는 단어들을 뽑아내기
nword = sorted(worddict.items(), key= lambda item : item[1], reverse = True)
ndict = dict(nword[:20]) #많이 나온 상위 20개 단어

#상위 20개의 단어들의 순서로 딕셔너리 제작
mapkey = dict([(val, idx) for idx, val in enumerate(list(ndict.keys()))]) #몇번째 인덱스인지 보려고 만드는 과정
mapkey

{',|SP': 0,
 '기술|NNG': 1,
 '주|NNG': 2,
 '증시|NNG': 3,
 '뉴욕|NNP': 4,
 '상승|NNG': 5,
 '에|JKM': 6,
 '…|SE': 7,
 '코로나|NNG': 8,
 '19|NR': 9,
 '도|JX': 10,
 '마감|NNG': 11,
 '%|SW': 12,
 '우려|NNG': 13,
 '강세|NNG': 14,
 '미국|NNP': 15,
 '테슬라|NNG': 16,
 '가|JKS': 17,
 "'|SS": 18,
 '반등|NNG': 19}

In [ ]:
wmap = [0 for i in range(20)]
wmap

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# word map 만들기

wordmap = []
for t in titles:
    kpos = kkma.pos(t)
    wmap = [0 for i in range(20)]  #상위 20개의 문자, 
    for pos in kpos:
        j ='|'.join(pos)
        if(j in mapkey.keys()):
            wmap[mapkey[j]] += 1
        else:
            pass
    wordmap.append(wmap)
print(wordmap)

[[2, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]


In [ ]:
#[2, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
#',|SP''기술|NNG''주|NNG''증시|NNG''뉴욕|NNP''상승|NNG''에|JKM''마감|NNG''…|SE''코로나|NNG''19|NR''도|JX''활약|NNG''%|SW''미국|NNP''테슬라|NNG''우려|NNG''강세|NNG''가|JKS''고|ECE'
    
    #이런 패턴이 나오면 검색량이 증가하고, 이런 패턴이 나오면 검색량이 하락하고 -> 가중치를 줌으로써 어떤 단어를 마케팅 단어로 사용해야 검색량이 증가하는지 볼 수 있다.

In [ ]:
from IPython.core import display as ICD
mapdata = pd.DataFrame(data=wordmap, columns = mapkey.keys())
ICD.display(mapdata)

,",|SP",기술|NNG,주|NNG,증시|NNG,뉴욕|NNP,상승|NNG,에|JKM,…|SE,코로나|NNG,19|NR,도|JX,마감|NNG,%|SW,우려|NNG,강세|NNG,미국|NNP,테슬라|NNG,가|JKS,'|SS,반등|NNG
0,2,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0
1,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0
4,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0,1,1,1,1,0,2,0,1,1,1,1,0,0,0,0,0,0,0,1
6,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
8,0,1,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0
9,2,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0
